<a href="https://colab.research.google.com/github/azahradwp/Menu-Recommendation-System/blob/main/Assignment_Legit_Group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ANSWER OF ASSIGNMENTS

In [64]:
#Import Library yang dibutuhkan

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

#
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
#reading file
path = '/content/drive/MyDrive/Dataset/data.csv'
df = pd.read_csv(path)
df = df.drop(columns=["trx_date", "trx_date_detail", "sales_id", "concept", "brand", "outlet", "district", "city"])
df.head(14)

,menu_id,menu_type,menu_category,menu_category_detail,menu_name,quantity,user_id,user_created_at,user_tier_level,user_gender
0,5204e17e-aefb-4235-8eb8-7177a5679352,Ala Carte,FOOD,APPETIZERS,French Fries,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
1,1420bcef-ac34-4967-b5e0-4a2942f222a4,Package Content,BEVERAGE,BEVERAGE MODIFIER,Single,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
2,1420bcef-ac34-4967-b5e0-4a2942f222a4,Package Content,BEVERAGE,BEVERAGE MODIFIER,Single,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
3,77bb78c6-2f51-4f61-9d74-02d69388cef1,Package Content,BEVERAGE,BEVERAGE MODIFIER,Coca Cola,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
4,f026874c-5a66-4184-8ce0-9aeb9e37b5ac,Package Content,LIQUEUR,WHISK(E)Y,Monkey Shoulder_Double,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
5,77bb78c6-2f51-4f61-9d74-02d69388cef1,Package Content,BEVERAGE,BEVERAGE MODIFIER,Coca Cola,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
6,77bb78c6-2f51-4f61-9d74-02d69388cef1,Package Content,BEVERAGE,BEVERAGE MODIFIER,Coca Cola,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
7,cfa23fc2-c4ab-4377-b0fe-489493016af1,Package Head,LIQUEUR,SINGLE MALT,Lagavulin 16 YO (Glass),1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
8,9708a976-0852-4705-a619-359447db7b09,Ala Carte,LIQUEUR,CLASSIC COCKTAIL,Lemon Drop,1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F
9,cfa23fc2-c4ab-4377-b0fe-489493016af1,Package Head,LIQUEUR,SINGLE MALT,Lagavulin 16 YO (Glass),1.0,db0df3a5-2d00-4fed-93b1-99ea1eedccd8,2021-05-13 07:41:27,Gold,F


In [67]:
#Data Exploration

df.isnull().sum()

menu_id                 0
menu_type               0
menu_category           0
menu_category_detail    0
menu_name               0
quantity                0
user_id                 0
user_created_at         0
user_tier_level         0
user_gender             0
dtype: int64

In [69]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26804 entries, 0 to 26803
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   menu_id               26804 non-null  object 
 1   menu_type             26804 non-null  object 
 2   menu_category         26804 non-null  object 
 3   menu_category_detail  26804 non-null  object 
 4   menu_name             26804 non-null  object 
 5   quantity              26804 non-null  float64
 6   user_id               26804 non-null  object 
 7   user_created_at       26804 non-null  object 
 8   user_tier_level       26804 non-null  object 
 9   user_gender           26804 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.0+ MB
None


#TASK 1 : Personalized Menu Recommendation

In [71]:
#Construct the User-Item Interaction Matrix

#Create a user-item interaction matrix
interaction_matrix = pd.pivot_table(df, values='quantity', index='user_id', columns='menu_name', fill_value=0)

#Convert the interaction matrix to a NumPy array
interaction_matrix_array = interaction_matrix.to_numpy()

#Print the interaction matrix
print(interaction_matrix)


menu_name                              ++Rich Syrup  \
user_id                                               
00b158ed-0fcb-48e1-b627-2bb572a344cf              0   
00bafc5b-3e97-439c-8b1f-9a6568adbaaa              0   
00cf8833-2cd5-40df-8477-3e9b73dbdb66              0   
0120f7c6-b3e7-4ff3-9082-90d311ca16f7              0   
014fc003-3f5b-4294-b2e6-836a17a0ed79              0   
...                                             ...   
fe476c7f-6392-43b4-887a-602372a7dfce              0   
fe8b2cb6-180e-443b-bb69-d44038034bd9              0   
ff13b265-54d4-48c4-8b51-f2e29e024e54              0   
ff39dbcb-4414-45b3-8fa0-92577d1c78b8              0   
ffde4412-810f-4800-a40b-27ad2a614a9e              0   

menu_name                              Truffle Butter Chicken Arrosto  \
user_id                                                                 
00b158ed-0fcb-48e1-b627-2bb572a344cf                                0   
00bafc5b-3e97-439c-8b1f-9a6568adbaaa                             

In [72]:
#Recommend items Function for a target user
def recommend_items_for_user(user_id, n=1):
    user_ratings = interaction_matrix.loc[user_id]
    recommended_items = np.argsort(user_ratings)[::-1][:n]
    return recommended_items

#Example: Recommend menu for target user '014fc003-3f5b-4294-b2e6-836a17a0ed79'
recommended_items = recommend_items_for_user('014fc003-3f5b-4294-b2e6-836a17a0ed79', n=1)
print("Recommended items for user:")
print(recommended_items)


Recommended items for user:
menu_name
“Mr.Fox Signature - Iced Tea”    1014
Name: 014fc003-3f5b-4294-b2e6-836a17a0ed79, dtype: int64


#TASK 2 : Customer Segmentation

In [76]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
!pip install KModes
from kmodes.kmodes import KModes
from sklearn.cluster import KMeans

#Dataset
data = df

#Preprocess data
selected_features = data[['user_tier_level', 'user_gender', 'quantity']]

#One-hot encode categorical variables
categorical_features = selected_features[['user_tier_level', 'user_gender']]
categorical_features_encoded = pd.get_dummies(categorical_features, drop_first=True)

#Scale numerical feature
numerical_feature = selected_features[['quantity']]
scaler = StandardScaler()
numerical_feature_scaled = scaler.fit_transform(numerical_feature)

#Combine encoded categorical features and scaled numerical feature
preprocessed_features = pd.concat([categorical_features_encoded, pd.DataFrame(numerical_feature_scaled, columns=['quantity'])], axis=1)

#Number of segments
num_segments = 4

#Train K-Modes model for categorical features
kmodes = KModes(n_clusters=num_segments, init='Huang', n_init=9, verbose=1)
categorical_clusters = kmodes.fit_predict(categorical_features_encoded)

#Train K-Means model for numerical features
kmeans = KMeans(n_clusters=num_segments, random_state=42)
numerical_clusters = kmeans.fit_predict(numerical_feature_scaled)

#Combine cluster labels
cluster_labels = categorical_clusters * num_segments + numerical_clusters

#Add cluster labels to the dataset
data['cluster'] = cluster_labels

#Define actionable treatments for each segment
segment_treatments = {
    0: "Personalized menu recommendations",
    1: "Special promotion for high-tier users",
    2: "Exclusive brand vouchers",
    3: "Tailored marketing campaigns"
}

#Assign treatments to each user based on their segment
data['treatment'] = data['cluster'].map(segment_treatments)

#Print out treatments for a target user
target_user_id = '1b9c05d6-f6da-43e1-8a0b-c72bcf8a5eff'
user_segment = data[data['user_id'] == target_user_id]['cluster'].values[0]
user_treatment = data[data['user_id'] == target_user_id]['treatment'].values[0]
print(f"User '{target_user_id}' belongs to segment {user_segment} and should receive treatment: {user_treatment}")


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3713, cost: 11800.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 0, cost: 15977.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 3215, cost: 11853.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 14962.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 2059, cost: 9703.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100, moves: 3215, cost: 11303.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 7, iteration: 1/100, moves: 2610, cost: 12687.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
